In [11]:
import numpy as np
import pandas as pd
from xclib.data import data_utils
import matplotlib.pylab as plt
from random import seed
from random import randrange
import math

In [45]:
# paths
data_folder_path = './data/'
train_x_path = data_folder_path + 'train_x.txt'
train_y_path = data_folder_path + 'train_y.txt'
test_x_path = data_folder_path + 'test_x.txt'
test_y_path = data_folder_path + 'test_y.txt'
val_x_path = data_folder_path + 'valid_x.txt'
val_y_path = data_folder_path + 'valid_y.txt'

In [3]:
def load_y(path, size):
    y = np.ones(size)
    f = open(path)
    cnt = 0
    for x in f:
        y[cnt] = int(x)
        cnt += 1
    return y

In [4]:
train_x = data_utils.read_sparse_file(train_x_path)
train_y = load_y(train_y_path, train_x.shape[0])

/home/karan/.local/lib/python3.6/site-packages/xclib-0.96-py3.6-linux-x86_64.egg/xclib/data/data_utils.py:173: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [5]:
test_x = data_utils.read_sparse_file(test_x_path)
test_y = load_y(test_y_path, test_x.shape[0])

In [6]:
val_x = data_utils.read_sparse_file(val_x_path)
val_y = load_y(val_y_path, val_x.shape[0])

In [7]:
val_x.shape

(21572, 482)

In [8]:
def median(data, feature):
    a = np.zeros(data.shape[1])
    cnt = 0
    for x in data:
        if(cnt==data.shape[1]):
            break
        a[cnt] = int(x.tolist()[0][feature])
#         print(x.tolist()[0][14])
        cnt += 1
    return np.median(a)

In [46]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            if(p>0):
                score -= p * (math.log2(p))
        # weight the group score by its relative size
        gini += (score) * (size / n_instances)
    return gini

def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
#             print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [47]:
math.log2(4.423)

2.1450252422365663

In [48]:
#train data clean
i = 0
train_data = np.zeros(shape=(64713, 483))
for x in train_x.toarray():
    x = np.append(x, train_y[i])
    train_data[i] = x
    i += 1
# train_data = train_data[:100]

In [32]:
#test data clean
i = 0
test_data = np.zeros(shape=(21571, 483))
for x in test_x.toarray():
    x = np.append(x, test_y[i])
    test_data[i] = x
    i += 1

In [33]:
#val data clean
i = 0
val_data = np.zeros(shape=(21572, 483))
for x in val_x.toarray():
    x = np.append(x, val_y[i])
    val_data[i] = x
    i += 1

In [ ]:
# global_depth = 1
tree = build_tree(train_data, 5, 10)

In [ ]:
tree

In [ ]:
predictions = list()
for row in train_data:
    prediction = predict(tree, row)
    predictions.append(prediction)

correct = 0
for i in range(len(predictions)):
    if(predictions[i] == train_data[i][-1]):
        correct += 1
print('Train Accuracy: ', correct/len(predictions))

# ====================================================================================
predictions = list()
for row in val_data:
    prediction = predict(tree, row)
    predictions.append(prediction)

correct = 0
for i in range(len(predictions)):
    if(predictions[i] == val_data[i][-1]):
        correct += 1
print('Val Accuracy: ', correct/len(predictions))

# ====================================================================================
predictions = list()
for row in test_data:
    prediction = predict(tree, row)
    predictions.append(prediction)

correct = 0
for i in range(len(predictions)):
    if(predictions[i] == test_data[i][-1]):
        correct += 1
print('Test Accuracy: ', correct/len(predictions))

In [50]:
# # evaluate algorithm
# n_folds = 2
# max_depth = 5
# min_size = 10
# scores = evaluate_algorithm(train_data, decision_tree, n_folds, max_depth, min_size)
# print('Scores: %s' % scores)
# print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Train Accuracy:  0.93
Val Accuracy:  0.6424995364361209
Test Accuracy:  0.6411385656668676